In [35]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
import requests
import pickle
from pathlib import Path
from datetime import datetime
from src.nlp.generate_lyrics import DeepLyric

## Text Generation

In [37]:
model = requests.get('https://storage.googleapis.com/w210-capstone/models/4.2-LM-108k-lines-genre-song_title_architecture.pkl')
model = model.content
model = pickle.loads(model)

In [38]:
itos = requests.get('https://storage.googleapis.com/w210-capstone/models/4.2-LM-108k-lines-genre-song_title_itos.pkl')
itos = itos.content
itos = pickle.loads(itos)

In [39]:
deep_lyric = DeepLyric(model, itos=itos)

In [49]:
deep_lyric.generate_text(seed_text='xbos xgenre', 
                         verbose=0, context_length=100, beam_width=3,
                         max_len=150, top_k=5, temperature=1.45,
                         GPU=True)

In [50]:
deep_lyric.print_lyrics()

SONG START
 genre: oldies 
 title: skeleton key (lp version ) 

xbol-1 whiskey just don 't bother me no more 
 honey , it 's mad when i don 't answer my telephone no more ... 
 train don 't surround me no more 
 traces won 't haunt me anymore 
 set me free baby , don 't bother me no more 
 honey won 't you rescue me baby don 't haunt me no more 
 girl don 't tell me no honey , babe no more 
 sugar don 't tell me that you 're leaving me now 
 
 chorus 
 won 't you shut me out 
 won 't you little baby won 't you say 
 won 't you darling won 't you hear me now 
 won 't you tell me why 
 won 

## Save to text file

In [93]:
num_iterations = 100
temp_ranges = [1.0, 1.2, 1.4, 1.6]
beam_ranges = [1, 2, 3, 4, 5]

#naming convention: i, temp, beam
OUT_PATH = Path(f'../data/lyrics/{model_name}')
OUT_PATH.mkdir(exist_ok=True)

for i in range(num_iterations):
    for temp in temp_ranges:
        for beam in beam_ranges:
            second_stamp = datetime.strftime(datetime.now(), '%s')
            f_name = f'../data/lyrics/{model_name}/{second_stamp}-{temp}-{beam}.txt'
            deep_lyric.generate_text(temperature=temp, beam_width=beam, GPU=True, verbose=0, max_len=150)
            deep_lyric.save_lyrics_to_file(f_name)

KeyboardInterrupt: 

In [94]:
OUT_STORE = f'gs://w210-capstone/lyrics/{model_name}/'

In [95]:
!gsutil -m -o cp $OUT_PATH/* $OUT_STORE

Copying file://../data/lyrics/4.2-LM-108k-lines-genre-song_title/1543790822-1.0-1.txt [Content-Type=text/plain]...
Copying file://../data/lyrics/4.2-LM-108k-lines-genre-song_title/1543790825-1.0-2.txt [Content-Type=text/plain]...
Copying file://../data/lyrics/4.2-LM-108k-lines-genre-song_title/1543790834-1.0-3.txt [Content-Type=text/plain]...
Copying file://../data/lyrics/4.2-LM-108k-lines-genre-song_title/1543790842-1.0-4.txt [Content-Type=text/plain]...
- [4 files][  2.7 KiB/  2.7 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m -o ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://../data/lyrics/4.2-LM-108k-lines-genre-song_title/1543790850-1.0-5.txt [Content-Type=text/plain]...
Copying file://../data/lyrics/4.2-LM-108k-lines-genre-song_title/1543790859-1.2-1.txt [Conte